# Busses

A typical application of a bulk signal is an interconnect bus that can be routed through several module hierarchies. To mimic the naming scheme of the **MaSoCist** SoC interface, we simply derive from the `ContainerGen` generator class and replace `._suffixes()`:


In [1]:
from cyhdl import *
from myirl.container import ContainerGen

class MasocistBusGen(ContainerGen):
    def _suffixes(self):
        return 'ReadPort', 'WritePort', 'AuxPort'

## Wishbone class

We define a Wishbone class as below. Also, simulation sequences are added to this class that are casted later.

In [2]:
@container(MasocistBusGen)
class WishBone:
    _inputs = [
        'dat', 'sel', 'ack', 'inta'
    ]
    _outputs = [
        'adr', 'dat', 'sel', 'cyc', 'stb', 'we', 'rst'
    ]
    _other  = [ 'clk', 'rst' ]
    
    def __init__(self, asize, dsize, context):
        Signal = context.Signal
        self.dat = Signal(modbv()[dsize:])
        self.adr = Signal(modbv()[asize:])
        self.sel, self.cyc, self.stb, self.we  = [ Signal(bool()) for _ in range(4) ]
        self.inta = Signal(bool())
        self.ack = Signal(bool(0))
        self.rst = context.ResetSignal(False, True)
        self.clk = context.ClkSignal()

    @cyrite_method.sequence
    def seq_write(self, addr, data):
        "Single bus write cycle"
        yield self.clk.negedge
        
        self.we.next = True
        self.adr.next = addr
        self.dat.next = data
        self.stb.next = True
        self.cyc.next = True
        yield self.clk.negedge
        self.stb.next = False
        self.cyc.next = False
        yield self.clk.posedge
        assert self.ack == True
        
    @cyrite_method.sequence
    def seq_read(self, addr):
        "Single bus read cycle"
        yield self.clk.negedge
        self.we.next = False
        self.adr.next = addr
        self.stb.next = True
        self.cyc.next = True
        yield self.clk.negedge
        self.stb.next = False
        self.cyc.next = False
        assert self.ack == True        

    @cyrite_method.sequence
    def seq_reset(self):
        "Bus reset"
        self.cyc.next = False
        self.stb.next = False
        yield [ self.clk.negedge, self.clk.posedge ]
        self.rst.next = True
        yield self.clk.posedge
        self.rst.next = False


Creating a wishbone instance for test:

In [3]:
import myirl
w = WishBone(asize = 15, dsize = 32, context = myirl)

The type definition of its children results in:

In [4]:
from myirl.targets.dummy import DummyVHDLModule
d = DummyVHDLModule()
for n, c in w.get_children().items():
    print("Emit type definition for %s:" % n)
    c.typedef(d)

Emit type definition for bulkc209_ReadPort:
type t_WishBone_15_32_obj_module_ReadPort is record
    dat : unsigned(31 downto 0);
    sel : std_ulogic;
    ack : std_ulogic;
    inta : std_ulogic;
end record;

Emit type definition for bulkc209_WritePort:
type t_WishBone_15_32_obj_module_WritePort is record
    adr : unsigned(14 downto 0);
    dat : unsigned(31 downto 0);
    sel : std_ulogic;
    cyc : std_ulogic;
    stb : std_ulogic;
    we : std_ulogic;
    rst : std_ulogic;
end record;

Emit type definition for bulkc209_AuxPort:
type t_WishBone_15_32_obj_module_AuxPort is record
    clk : std_ulogic;
    rst : std_ulogic;
end record;



# A Wishbone 'consumer'

A top level bus slave unit. We insert some monitor signals, as the VCD trace does not support tracing records. Using the `.ghw` wave format (--wave option of GHDL) this is supported by the GTKwave display (not within the browser).

In [5]:
@block
def top(bus : WishBone.reverse()):
    
    mon_adr = bus.adr.alias("adr")
    mon_dat = bus.dat.alias("data")
    mon_ack = bus.ack.alias("ack")
    mon_cyc = bus.cyc.alias("cyc")
    mon_rst = bus.rst.alias("rst")

    # Currently we need to explicitely wire them up:
    wires = [
        mon_adr.wireup(bus.adr),
        mon_dat.wireup(bus.dat),
        mon_rst.wireup(bus.rst),
        mon_cyc.wireup(bus.cyc)
    ]
   
    @always(delay(2))
    def bus_ack(ctx = None):
        bus.ack.next = mon_ack
    
    # Explicitely insert unused monitor signals:
    top.insert(mon_adr, mon_dat, mon_ack, mon_cyc, mon_rst)
    
    @always_seq(bus.clk.posedge, bus.rst)
    def worker():
        mon_ack.next = bus.cyc          
        
    @always(bus.clk.posedge)
    def debugger():
        "This just prints the data received upon each clock's rising edge"
        if bus.cyc == True:
            print("ACK data", mon_dat)
    
    return instances()

### The test bench

We create another derivative with the write_sequence extension below. This is a HDL macro, because we explicitely want to generate a sequence. The for loop is not emitted, but the yields inside it are unrolled.

In [6]:
from myirl.kernel.components import Comment
from myirl import simulation

class WBX(WishBone):
    comment = Comment
    @hdlmacro
    def write_sequence(self, mclk, addr, values):
        "Function unrolling a value sequence"
        yield [ self.comment("Write sequence") ]
        for i, e in enumerate(values):
            yield [ self.comment("Run %d" % i) ]
            yield [ self.seq_write(addr + i, e) ]
            yield 3 * (simulation.wait(mclk.posedge), )
            

Next, we call this sequence writer using `.evaluate()` in the test bench below.

In [7]:
from myirl.test.common_test import gen_osc

class MyDesign(cyrite_factory.Module):
    
    @cyrite_factory.testbench('ns')
    def tbwb(self):
        reset = self.ResetSignal(False, True)
        
        mclk = self.ClkSignal(name = "mclk")
        
        # Bus instance on the top level
        bus = WBX(12, 16, name = "wb0", context = self)
        
        clkgen = gen_osc(mclk, CYCLE=5)
        
        wires = [
            bus.clk.wireup(mclk)
        ]

        inst_top = top(bus)
        
        values = [ 0xdead, 0xbeef, 0xf00d, 0xface ]

        @self.always(delay(2))
        def clkgen():
            mclk.next = ~mclk
        
        @self.sequence
        def seq():
            "Bus master stimuli"
            yield delay(20)
            yield from bus.seq_reset()
            # This is the portable way to call an explicit hdlmacro:
            bus.write_sequence(mclk, 0x800, values).evaluate(seq)
            # Valid for HDL target simulation, but not portable to co-simulation:
            # yield from bus.write_sequence(mclk, 0x800, values).evaluate(seq)

    
            raise StopSimulation
            
        return instances()

In [8]:
from cyrite.simulation import ghdl

def test():     
    m = MyDesign("mine", ghdl.GHDL)
    tb = m.tbwb()
    tb.run(2000, wavetrace = "tbwb.vcd", debug = True)
    return tb
    
tb = test()

 Declare obj 'tbwb' in context '(MyDesign 'mine')'(<class '__main__.MyDesign'>) 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_ReadPort 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_WritePort 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_AuxPort 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_ReadPort 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_WritePort 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_AuxPort 
 DEBUG REGISTER mine: WBX_12_16_obj_MyDesign_WritePort 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_WritePort 
 DEBUG REGISTER mine: WBX_12_16_obj_MyDesign_AuxPort 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_AuxPort 
 DEBUG REGISTER mine: WBX_12_16_obj_MyDesign_ReadPort 
 DEBUG REGISTER module_defs: WBX_12_16_obj_MyDesign_ReadPort 
 Writing 'top' to file /tmp/top.vhdl 
 Writing 'tbwb' to file /tmp/tbwb.vhdl 
 Creating library file /tmp/module_defs.vhdl 
DEBUG_FILES ['/tmp/top.vhdl', '/tmp/tbwb.vhdl', '/tmp/module_defs.vhdl',

In [9]:
!cat /tmp/tbwb.vhdl

-- File generated from source:
--     /tmp/ipykernel_95960/124006512.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.module_defs.all;
use work.txt_util.all;
use work.myirl_conversion.all;

entity tbwb is
end entity tbwb;

architecture irl_uncached of tbwb is
    -- Local type declarations
    -- Signal declarations
    signal mclk : std_ulogic := '0';
    signal wb0_WritePort : t_WBX_12_16_obj_MyDesign_WritePort;
    signal wb0_AuxPort : t_WBX_12_16_obj_MyDesign_AuxPort := (clk => '0', rst => '0');
    signal wb0_ReadPort : t_WBX_12_16_obj_MyDesign_ReadPort;
begin
    
clkgen:
    process(mclk)
    begin
        mclk <= not mclk after 2.000000 ns;
    end process;

    
    -- Instance top
    inst_top_0: entity work.top
    port map (
        bus_wb0_ReadPort => wb0_ReadPort,
        bus_wb0_WritePort => wb0_WritePort,
        bus_wb0_AuxPort => wb

### Waveform display

Again a timing-inaccurate waveform display, just sufficient for explaining the cycles:

In [10]:
clkname = 'mclk'

from cyrite import waveutils
waveutils.draw_wavetrace(tb, 'tbwb.vcd', clkname)